In [41]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-29 20:47:59--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.20’

minsearch.py.20     100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-29 20:47:59 (44.3 MB/s) - ‘minsearch.py.20’ saved [3832/3832]



In [42]:
import minsearch

In [43]:
import json

In [44]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [45]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']: 
        doc['course'] = course_dict['course']
        documents.append(doc)

In [46]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [47]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [48]:
q = 'the course has already started, can I still enroll?'

In [49]:
index.fit(documents)

In [50]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict = boost, 
    num_results=5
)

In [51]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [52]:
!pip install openai

import openai

import os


In [53]:
from openai import OpenAI

In [54]:
client = OpenAI()

In [55]:
q

'the course has already started, can I still enroll?'

In [56]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [57]:
response.choices[0].message.content

"It's possible that you may still be able to enroll in a course after it has started, but this depends on the policies of the institution or organization offering the course. Here are a few steps you can take:\n\n1. **Check Enrollment Policies**: Look up the specific course or institution's enrollment policies on their website. Some courses may allow late enrollment within a certain timeframe.\n\n2. **Contact the Instructor or Administration**: Reach out to the course instructor or the academic administration to explain your situation and inquire about the possibility of late enrollment. They may be able to make an exception or provide guidance on the next steps.\n\n3. **Catch Up on Missed Content**: If you are allowed to enroll late, be prepared to catch up on any missed lectures, assignments, and readings to ensure you don't fall behind.\n\n4. **Consider Online Options**: Some online courses have flexible start dates or are available on demand, which could provide more flexibility if

In [58]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()

In [59]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [60]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [61]:
prompt = prompt_template.format(question=q, context=context).strip()

In [62]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [63]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [64]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost, 
        num_results=5
    )

    return results

In [65]:
query = 'how do I run kafka?'
search_results = search(query)

In [66]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [67]:
prompt = build_prompt(query, search_results)

In [68]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [69]:
query = 'how do I run kafka?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [70]:
answer

'To run Kafka, here are different approaches based on whether you are using Java or Python as per our course context:\n\n### For Java Kafka:\n\n1. **Running Producer/Consumer/KStreams in Terminal:**\n   Navigate to your project directory and execute the following command:\n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n### For Python Kafka:\n\n1. **Setting Up a Virtual Environment:**\n   First, create and activate a virtual environment where you will install the required dependencies.\n   ```bash\n   python -m venv env\n   source env/bin/activate   # MacOS/Linux\n   # or \n   env\\Scripts\\activate      # Windows\n   ```\n\n2. **Installing Dependencies:**\n   Run the command:\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\n3. **Running Your Python Kafka Producer:**\n   Once the virtual environment is set up with the necessary packages, you can run your producer script.\n\nRemember, for Docker envir

In [71]:
search_results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [72]:
query = 'How do I execute a command in a running docker container?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [73]:
rag(query)

'To execute a command in a running Docker container, you can use the `docker exec` command. Here is the format:\n\n```bash\ndocker exec -it <container_name> <command>\n```\n\nFor instance, if you have a running container named `pg-database` and you want to execute the `pgcli` command inside it, you would do something like this:\n\n```bash\ndocker exec -it pg-database pgcli -h pg-database -U root -p 5432 -d ny_taxi\n```\n\nReplace `<container_name>` with the name of your Docker container and `<command>` with the command you wish to execute within the container. In the given `pgcli` example, it connects to a PostgreSQL database named `ny_taxi`.'

In [74]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [75]:
from elasticsearch import Elasticsearch

In [76]:
es_client = Elasticsearch('http://localhost:9200')

In [77]:
es_client.info()

ObjectApiResponse({'name': '3d934389e4b7', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'e-d_rMI7SWKM4t3S5QYYIg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [78]:
index_name = "course-question"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)


In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question"

es_client.indices.create(index=index_name, body=index_settings)
    

In [79]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

tqdm is for progress bar
using the tqdm library to add a progress bar to your loop over documents. This can be helpful for tracking the progress of your loop, especially if documents is a large iterable.

In [80]:
from tqdm import tqdm

In [81]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.36it/s]


In [82]:
query = 'How do I execute a command in a running docker container?'

In [83]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [84]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
    

In [99]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# 將Elasticsearch中返回的記錄轉換為上下文格式
context = ""
for doc in results:
    context += context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

# 使用模板構建最終提示
prompt = prompt_template.format(question="How do I execute a command in a running docker container?", context=context)

# 查看最終提示的長度
prompt_length = len(prompt)
print(prompt_length)


2136


In [100]:
elastic_search(query)

[]

In [101]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: Course - Can I still join the course after the start date?
A: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Q: Course - Can I follow the course after it finishes?
A: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

Q: Course - When will the course start?
A: The purpose of this document is to capture frequently asked technical questions
The exact da

In [87]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [88]:
rag(query)

'To execute a command in a running Docker container, you can use the following command:\n\n```sh\ndocker exec [OPTIONS] CONTAINER COMMAND [ARG...]\n```\n\nFor example, to run a command like `ls` inside a container named `my_container`, you would use:\n\n```sh\ndocker exec my_container ls\n```\n\nYou can also start an interactive terminal session inside a running container using:\n\n```sh\ndocker exec -it my_container /bin/bash\n```\n\nThis will open a bash shell in the container, allowing you to run commands interactively.'

In [89]:
import tiktoken

In [96]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
prompt = 

In [98]:
print(encoding)

<Encoding 'o200k_base'>


In [90]:
import sys
print(sys.executable)

/home/codespace/miniconda3/bin/python


In [91]:
!which python

/home/codespace/miniconda3/bin/python


In [92]:
!conda env list

# conda environments:
#
base                  *  /home/codespace/miniconda3
                         /opt/conda



In [93]:
import sys; print(sys.executable)

/home/codespace/miniconda3/bin/python


In [94]:
import sys; sys.path.append('/home/codespace/miniconda3/bin/python')

In [95]:
import sys; print(sys.executable)

/home/codespace/miniconda3/bin/python
